In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
from datetime import datetime, timedelta
import time

In [2]:
# data_inicio = '01/01/2000'
# data_fim = '28/08/2024'

data_inicio = (datetime.today() - timedelta(days=7)).strftime('%d/%m/%Y')
data_fim = datetime.today().strftime('%d/%m/%Y')

#libra, dolar, renmimbi chines, novo dolar taiwan, euro
codigo = [115, 61, 178, 141, 222]
dataframes = []

for i in codigo:
    url = f'https://ptax.bcb.gov.br/ptax_internet/consultaBoletim.do?method=gerarCSVFechamentoMoedaNoPeriodo&ChkMoeda={i}&DATAINI={data_inicio}&DATAFIM={data_fim}'
    response = requests.get(url)

    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data, delimiter=';', names=["data", "codigo", "tipo", "sigla", 'compra_real','venda_real', 'compra_dolar', 'venda_dolar'])

    dataframes.append(df)

In [3]:
df_libra = dataframes[0]
df_dolar = dataframes[1]
df_renmimbi = dataframes[2]
df_taiwan = dataframes[3]
df_euro = dataframes[4]

In [4]:
df_dolar_columns = df_dolar[['data', 'compra_real', 'venda_real']]

In [5]:
df_dolar_rename = df_dolar_columns.rename(columns={
      'compra_real':f'compra_dolar',
      'venda_real':f'venda_dolar'})

In [6]:
df_dolar_rename.loc[:, 'data'] = df_dolar_rename['data'].astype(str).str.zfill(8)

In [7]:
df_dolar_rename['compra_dolar'] = df_dolar_rename['compra_dolar'].str.replace(',', '.')
df_dolar_rename['compra_dolar'] = df_dolar_rename['compra_dolar'].astype(float)

In [8]:
df_dolar_rename['venda_dolar'] = df_dolar_rename['venda_dolar'].str.replace(',', '.')
df_dolar_rename['venda_dolar'] = df_dolar_rename['venda_dolar'].astype(float)

In [9]:
df_dolar_rename['data'] = pd.to_datetime(df_dolar_rename['data'], format='%d%m%Y')
df_dolar_final = df_dolar_rename

In [10]:
def ajuste(df, moeda):

  df = df[['data', 'compra_real', 'venda_real', 'compra_dolar', 'venda_dolar']].copy()

  df['compra_real'] = df['compra_real'].str.replace(',', '.')
  df['venda_real'] = df['venda_real'].str.replace(',', '.')
  df['compra_dolar'] = df['compra_dolar'].str.replace(',', '.')
  df['venda_dolar'] = df['venda_dolar'].str.replace(',', '.')

  df['compra_real'] = df['compra_real'].astype(float)
  df['venda_real'] = df['venda_real'].astype(float)
  df['compra_dolar'] = df['compra_dolar'].astype(float)
  df['venda_dolar'] = df['venda_dolar'].astype(float)

  df = df.rename(columns={
      'compra_real':f'compra_real_{moeda}',
      'venda_real':f'venda_real_{moeda}',
      'compra_dolar':f'compra_dolar_{moeda}',
      'venda_dolar':f'venda_dolar_{moeda}'})

  df.loc[:, 'data'] = df['data'].astype(str).str.zfill(8)

  df['data'] = pd.to_datetime(df['data'], format='%d%m%Y')

  return df

In [11]:
df_libra_final = ajuste(df_libra, 'libra')
df_renmimbi_final = ajuste(df_renmimbi, 'renmimbi')
df_taiwan_final = ajuste(df_taiwan, 'taiwan')
df_euro_final = ajuste(df_euro, 'euro')

In [12]:
df_libra_final.set_index('data', inplace=True)
df_renmimbi_final.set_index('data', inplace=True)
df_taiwan_final.set_index('data', inplace=True)
df_dolar_final.set_index('data', inplace=True)
df_euro_final.set_index('data', inplace=True)

In [13]:
df_combined = pd.concat([df_dolar_final, df_libra_final, df_renmimbi_final, df_taiwan_final, df_euro_final], axis=1)

df_combined.reset_index(inplace=True)

In [14]:
df_combined

,data,compra_dolar,venda_dolar,compra_real_libra,venda_real_libra,compra_dolar_libra,venda_dolar_libra,compra_real_renmimbi,venda_real_renmimbi,compra_dolar_renmimbi,venda_dolar_renmimbi,compra_real_taiwan,venda_real_taiwan,compra_dolar_taiwan,venda_dolar_taiwan,compra_real_euro,venda_real_euro,compra_dolar_euro,venda_dolar_euro
0,2024-09-09,5.6091,5.6097,7.3378,7.3392,1.3082,1.3083,0.7885,0.7886,7.1133,7.1140,0.1746,0.1748,32.095,32.125,6.1930,6.1942,1.1041,1.1042
1,2024-09-10,5.6248,5.6254,7.3482,7.3496,1.3064,1.3065,0.7899,0.7900,7.1205,7.1210,0.1747,0.1749,32.171,32.201,6.2008,6.2026,1.1024,1.1026
2,2024-09-11,5.6381,5.6387,7.3425,7.3438,1.3023,1.3024,0.7919,0.7920,7.1192,7.1199,0.1754,0.1756,32.120,32.150,6.2087,6.2099,1.1012,1.1013
3,2024-09-12,5.6548,5.6554,7.3965,7.3978,1.3080,1.3081,0.7943,0.7946,7.1173,7.1195,0.1757,0.1759,32.148,32.178,6.2429,6.2441,1.1040,1.1041
4,2024-09-13,5.5711,5.5717,7.3199,7.3212,1.3139,1.3140,0.7853,0.7855,7.0930,7.0944,0.1743,0.1745,31.929,31.959,6.1750,6.1768,1.1084,1.1086


In [ ]:
df_combined = df_combined.sort_values(by='data')

df_combined.to_excel(f'Base_Moedas.xlsx', index=False, sheet_name= 'Base_Moedas')